In [1]:
!pip install torch torchvision matplotlib

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.3/2.3 MB 20.0 MB/s eta 0:00:00


In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import datasets, transforms
from torch.utils.data import DataLoader
import matplotlib.pyplot as plt

# 1. Hyperparameters and Device Setup
# A 'hyperparameter' is a setting that controls the training process.
batch_size = 64
learning_rate = 0.001
epochs = 5
device = "cuda" if torch.cuda.is_available() else "cpu"
print(f"Using {device} device")

# 2. Data Loading and Preprocessing
# We use torchvision to download and prepare the MNIST dataset.
# The `transform` resizes images to a tensor and normalizes their pixel values.
transform = transforms.ToTensor()

train_dataset = datasets.MNIST(
    root="./data", train=True, download=True, transform=transform
)
test_dataset = datasets.MNIST(
    root="./data", train=False, download=True, transform=transform
)

# DataLoader creates batches and shuffles the data for us.
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)

# 3. Define the Neural Network Model
# We create a simple, multi-layered neural network.
class NeuralNetwork(nn.Module):
    def __init__(self):
        super().__init__()
        self.flatten = nn.Flatten()
        self.stack = nn.Sequential(
            nn.Linear(28 * 28, 512),  # Input layer (28*28 pixels) -> Hidden layer
            nn.ReLU(),                # Activation function
            nn.Linear(512, 512),      # Another hidden layer
            nn.ReLU(),                # Activation function
            nn.Linear(512, 10)        # Output layer (10 classes for digits 0-9)
        )

    def forward(self, x):
        x = self.flatten(x)
        logits = self.stack(x)
        return logits

model = NeuralNetwork().to(device)
print(model)

# 4. Define Loss Function and Optimizer
# The loss function measures how wrong our model's predictions are.
# The optimizer updates the model's weights to minimize the loss.
loss_fn = nn.CrossEntropyLoss()
optimizer = optim.SGD(model.parameters(), lr=learning_rate)

# 5. Training Function
def train(dataloader, model, loss_fn, optimizer):
    model.train() # Set model to training mode
    total_loss = 0
    for batch, (X, y) in enumerate(dataloader):
        X, y = X.to(device), y.to(device)

        # Forward pass: get predictions
        pred = model(X)
        loss = loss_fn(pred, y)

        # Backward pass: update weights
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        total_loss += loss.item()

    avg_loss = total_loss / len(dataloader)
    print(f"  Training Loss: {avg_loss:.4f}")

# 6. Testing Function
def test(dataloader, model, loss_fn):
    model.eval() # Set model to evaluation mode
    test_loss, correct = 0, 0
    with torch.no_grad():
        for X, y in dataloader:
            X, y = X.to(device), y.to(device)
            pred = model(X)
            test_loss += loss_fn(pred, y).item()
            correct += (pred.argmax(1) == y).type(torch.float).sum().item()

    test_loss /= len(dataloader)
    correct /= len(dataloader.dataset)
    print(f"  Test Accuracy: {(100*correct):>0.1f}%, Test Loss: {test_loss:>8f}")

# 7. Main Training Loop
print("\nStarting training...")
for t in range(epochs):
    print(f"Epoch {t+1}")
    train(train_loader, model, loss_fn, optimizer)
    test(test_loader, model, loss_fn)
print("Training Complete!")

Using cpu device


100%|████████████████████████████████████████████████████████████████████████████████| 9.91M/9.91M [00:04<00:00, 2.14MB/s]
100%|█████████████████████████████████████████████████████████████████████████████████| 28.9k/28.9k [00:00<00:00, 385kB/s]
100%|████████████████████████████████████████████████████████████████████████████████| 1.65M/1.65M [00:00<00:00, 1.68MB/s]
100%|████████████████████████████████████████████████████████████████████████████████| 4.54k/4.54k [00:00<00:00, 5.41MB/s]


NeuralNetwork(
  (flatten): Flatten(start_dim=1, end_dim=-1)
  (stack): Sequential(
    (0): Linear(in_features=784, out_features=512, bias=True)
    (1): ReLU()
    (2): Linear(in_features=512, out_features=512, bias=True)
    (3): ReLU()
    (4): Linear(in_features=512, out_features=10, bias=True)
  )
)

Starting training...
Epoch 1
  Training Loss: 2.2811
  Test Accuracy: 44.9%, Test Loss: 2.258112
Epoch 2
  Training Loss: 2.2328
  Test Accuracy: 64.6%, Test Loss: 2.198161
Epoch 3
  Training Loss: 2.1563
  Test Accuracy: 68.8%, Test Loss: 2.095672
Epoch 4
  Training Loss: 2.0209
  Test Accuracy: 70.8%, Test Loss: 1.914043
Epoch 5
  Training Loss: 1.7923
  Test Accuracy: 71.9%, Test Loss: 1.629750
Training Complete!
